# EEG data classification Guinnea Bissau

This notebook contains experiments with an EEG dataset. The classes are Epilepsy: 0 or Control 1.

Load dependences and setting output configuration

In [1]:
import numpy as np
from keras.utils.np_utils import to_categorical
import keras
%pylab inline
%load_ext autoreload
%autoreload 2

Using Theano backend.


Populating the interactive namespace from numpy and matplotlib


## Load data from npy files

Specify location of npy files:

In [2]:
datapath = '/home/sonja/EEGs_Guinea-Bissau_np/'
#datapath = '/media/sf_VBox_Shared/timeseries/EEGs_Guinea-Bissau_np/'#

Load data stored in 10 seconds at 128 Hertz corresponding to the experiment where the participant had the eyes closed:

In [42]:
condition = '_10seconds_closed.npy'
X_train = np.load(datapath+'X_train'+condition)
y_train = np.load(datapath+'y_train'+condition)
X_val = np.load(datapath+'X_valid'+condition)
y_val = np.load(datapath+'y_valid'+condition)
X_test = np.load(datapath+'X_test'+condition)
y_test = np.load(datapath+'y_test'+condition)

In [43]:
print(X_train)

[[[ 4184.1025641   3924.1025641   4216.92307692 ...,  4030.25641026
    3809.74358974  3867.17948718]
  [ 4180.51282051  3913.33333333  4213.33333333 ...,  4021.53846154
    3801.02564103  3858.97435897]
  [ 4181.02564103  3915.38461538  4216.41025641 ...,  4023.07692308
    3799.48717949  3860.        ]
  ..., 
  [ 4266.66666667  3867.69230769  4250.76923077 ...,  4061.53846154
    3802.56410256  3911.79487179]
  [ 4265.64102564  3876.92307692  4249.23076923 ...,  4061.02564103
    3800.51282051  3911.79487179]
  [ 4268.71794872  3878.97435897  4250.25641026 ...,  4062.05128205  3800.
    3912.30769231]]

 [[ 4352.82051282  3938.97435897  4187.69230769 ...,  4175.8974359
    3862.56410256  3964.1025641 ]
  [ 4347.69230769  3928.71794872  4185.12820513 ...,  4163.07692308
    3861.02564103  3953.84615385]
  [ 4353.84615385  3923.58974359  4197.43589744 ...,  4153.33333333
    3863.07692308  3945.64102564]
  ..., 
  [ 4188.20512821  3933.84615385  4209.74358974 ...,  4172.30769231
    3

In [11]:
# rather than the original time-series, use the jumps. Makes the time-series stationary
# X is a time-series of n(=14) channels
def differences(X):
    dim1 = len(X)-1
    dim2 = len(X[0])
    M = np.zeros((dim1, dim2))
    #M = X
    for i in range(0, dim1-1):
        for j in range (0,dim2-1):
            M[i][j] = X[i+1][j] - X[i][j]
    return M            
            

In [ ]:
differences(X_train[0])

In [13]:
# substracts m from the whole dataset
def substract_mean(X,m):
    image = []
    for i in range (0, len(X -1)):
        X[i]-=m        
        image.append(X[i])
    return np.array(image) 

In [30]:
# converts the whole dataset X by applying diffenences function to individuals
def convertDataset(X):
    image = []
    for i in range (0, len(X -1)):
        tr = differences(X[i])
        image.append(tr)
    return np.array(image)    
    

In [31]:
# transposes the dataset of each individual
def transposeDataset(X):
    image = []
    for i in range (0, len(X -1)):
        tr = X[i].transpose()
        image.append(tr)
    return np.array(image)

In [44]:
m = np.mean(X_train)
image1 = substract_mean(X_train,m)
image2 = substract_mean(X_val,m)
image3 = substract_mean(X_test,m)
X_train = image1
X_val = image2
X_test = image3

In [45]:
image1 = convertDataset(X_train)
image2 = convertDataset(X_val)
image3 = convertDataset(X_test)
X_train = image1
X_val = image2
X_test = image3

In [46]:
image1 = transposeDataset(X_train)
image2 = transposeDataset(X_val)
image3 = transposeDataset(X_test)
X_train = image1
X_val = image2
X_test = image3

In [47]:
print(X_train[1])

[[ -5.12820513   6.15384615   4.1025641  ...,   7.17948718  -4.61538462
    0.        ]
 [-10.25641026  -5.12820513   2.05128205 ...,   9.74358974   0.51282051
    0.        ]
 [ -2.56410256  12.30769231   7.17948718 ...,   5.12820513  -2.56410256
    0.        ]
 ..., 
 [-12.82051282  -9.74358974  -3.58974359 ...,   8.71794872  -5.64102564
    0.        ]
 [ -1.53846154   2.05128205   0.51282051 ...,   2.05128205   3.58974359
    0.        ]
 [  0.           0.           0.         ...,   0.           0.           0.        ]]


In [18]:
classlabels = list(set(y_train))
mapclasses = {classlabels[i] : i for i in range(len(classlabels))}
print(mapclasses)

{'Epilepsy': 0, 'Control': 1}


In [19]:
y_train = np.array([mapclasses[c] for c in y_train], dtype='int')
y_val = np.array([mapclasses[c] for c in y_test], dtype='int')
y_test = np.array([mapclasses[c] for c in y_test], dtype='int')
y_train_binary = to_categorical(y_train)
y_val_binary = to_categorical(y_val)
y_test_binary = to_categorical(y_test)

In [20]:
y_val_binary

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

## Generate models

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D, Flatten, MaxPooling1D
from keras.optimizers import Adam
import sys
import os
import numpy as np
sys.path.insert(0, os.path.abspath('..'))
from mcfly import modelgen, find_architecture

In [22]:
np.random.seed = 321
num_classes = y_train_binary.shape[1]

In [54]:
%%time
models = modelgen.generate_models(X_train.shape,
                                  num_classes,
                                  number_of_models = 5,
                                  model_type = 'CNN',
                                  cnn_min_layers=2,
                                  cnn_max_layers=4,
                                 low_lr=2, high_lr=8)

CPU times: user 960 ms, sys: 0 ns, total: 960 ms
Wall time: 1.24 s


In [56]:
%%time
for model, params, model_types in models:
    print(params)
    model.summary()

{'learning_rate': 0.0018468900521431515, 'regularization_rate': 0.013332580745925337, 'fc_hidden_nodes': 1734, 'filters': array([79, 15])}
____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
batchnormalization_81 (BatchNormali(None, 14, 1279)    2558        batchnormalization_input_16[0][0]
____________________________________________________________________________________________________
convolution1d_51 (Convolution1D)   (None, 14, 79)      303202      batchnormalization_81[0][0]      
____________________________________________________________________________________________________
batchnormalization_82 (BatchNormali(None, 14, 79)      158         convolution1d_51[0][0]           
____________________________________________________________________________________________________
activation_81 (Activation)         (None, 14, 79)    

## Compare models

Currently run with a very low number of epochs

In [55]:
%%time
histories, val_accuracies, val_losses = find_architecture.train_models_on_samples(X_train, y_train_binary,
                                                                                 X_val, y_val_binary,
                                                                                 models,nr_epochs=15,
                                                                                  subset_size=200,
                                                                                  verbose=True)

Training model 0 CNN
Train on 108 samples, validate on 20 samples
Epoch 1/15
108/108 [==============================] - 0s - loss: 24.0027 - acc: 0.4537 - val_loss: 1.7179 - val_acc: 0.4000
Epoch 2/15
108/108 [==============================] - 0s - loss: 20.2835 - acc: 0.8611 - val_loss: 1.0840 - val_acc: 0.4000
Epoch 3/15
108/108 [==============================] - 0s - loss: 16.8971 - acc: 0.9259 - val_loss: 0.9167 - val_acc: 0.3000
Epoch 4/15
108/108 [==============================] - 1s - loss: 13.7490 - acc: 0.9722 - val_loss: 0.8416 - val_acc: 0.3500
Epoch 5/15
108/108 [==============================] - 0s - loss: 11.0314 - acc: 0.9537 - val_loss: 0.8185 - val_acc: 0.2500
Epoch 6/15
108/108 [==============================] - 0s - loss: 8.7693 - acc: 0.9815 - val_loss: 0.7938 - val_acc: 0.3000
Epoch 7/15
108/108 [==============================] - 0s - loss: 6.9296 - acc: 1.0000 - val_loss: 0.7786 - val_acc: 0.3500
Epoch 8/15
108/108 [==============================] - 0s - loss: 5.5

In [ ]:
%%time
for i in range(len(models)):
    name = str(models[i][1])
    find_architecture.plotTrainingProcess(histories[i], name)

In [ ]:
#%%time
import pandas as pd
results = pd.DataFrame({'model':[str(params) for model, params, model_types in models],
                       'train_acc': [history.history['acc'][-1] for history in histories],
                       'train_loss': [history.history['loss'][-1] for history in histories],
                       'val_acc': [history.history['val_acc'][-1] for history in histories],
                       'val_loss': [history.history['val_loss'][-1] for history in histories]
                       })
results

## Train the best model for real

In [ ]:
best_model_index = np.argmax(val_accuracies)
#best_model_index = np.argmax(val_losses)
#best_model_index = 1

In [ ]:
import theano
theano.config.mode

In [ ]:
best_model, best_params, best_model_types = models[best_model_index]
print(best_model_index, best_model_types, best_params)

In [ ]:
best_model_copy = modelgen.generate_CNN_model(X_train.shape, num_classes, best_params['filters'], best_params['fc_hidden_nodes'],
                       best_params['learning_rate'], best_params['regularization_rate'])
best_model_copy = best_model
print(best_model_index, best_model_types, best_params)

In [ ]:
nr_epochs = 10
datasize = X_train.shape[0]#1000
history = best_model_copy.fit(X_train[:datasize,:,:], y_train_binary[:datasize,:],
              nb_epoch=nr_epochs, validation_data=(X_val, y_val_binary), batch_size=20)

In [ ]:
find_architecture.plotTrainingProcess(history)

## Inspect model predictions

In [ ]:
datasize = X_val.shape[0]
best_model_copy.predict_proba(X_val[:datasize,:,:],batch_size=1)

In [ ]:
best_model_copy.summary()

In [ ]:
from keras import backend as K

# with a Sequential model
get_dens_layer_output = K.function([best_model_copy.layers[0].input, K.learning_phase()],
                                  [best_model_copy.layers[0].output])
layer_output = get_dens_layer_output([X_val, 0])[0]

In [ ]:
layer_output.shape

In [ ]:
layer_output.mean(axis=(0,1))

In [ ]:
layer = best_model.layers[0]
for w in layer.get_weights():
    print(w.shape)

## See if we can overfit on a small train set

In [ ]:
params = models[0][1]
print(params)
small_model = modelgen.generate_CNN_model(X_train.shape, num_classes, params['filters'], params['fc_hidden_nodes'],
                                  0.01, #params['learning_rate'], 
                                        regularization_rate=0)
small_model.summary()

In [ ]:
small_model.evaluate(X_val, y_val_binary)

In [ ]:
nr_epochs = 100
datasize = 20
history = small_model.fit(X_train[:datasize,:,:], y_train_binary[:datasize,:],
              nb_epoch=nr_epochs, validation_data=(X_val, y_val_binary), batch_size=10)

## Test on Testset

In [ ]:
score_test = best_model.evaluate(X_test, y_test_binary, verbose=False)
print('Score of best model: ' + str(score_test))

In [ ]:
best_model.get_config()[0]